<hr style="height:2px;border:none"/>
<h1 align='center'> INF-395/477 Redes Neuronales Artificiales I-2018 </h1>

<H3 align='center'> Tarea 2 - Aplicaciones Recientes de Redes Neuronales </H3>
<hr style="height:2px;border:none"/>

<H3 align='center'> Fernanda Urrea, ROL: 201551522-0</H3>
<H3 align='center'>Matías Gómez, ROL: 201460501-3</H3>

## 3. *Encoder-Decoder* sobre Texto

Trabajos recientes en redes neuronales han demostrado que se puede aplicar a problemas bastante complejos gracias a la flexibilidad la definición de las redes, además de que se pueden adaptar a distintos tipos de datos brutos (dominios). Con el objetivo de explorar el enfoque anterior de *traducción* de algun tipo de dato, en esta sección deberá realizarlo con texto para traducción de un lenguaje humano a otro (e.g. inglés a alemán, chino a ruso).

<img src="https://www.panoramaila.cz/images/preklady.jpg" width="35%" />


Trabajaremos con el dataset de pares de sentencias bilingues entre distintos idiomas del proyecto __[Tatoeba](http://www.manythings.org/anki/)__. El objetivo entonces consta de tomar un texto en lenguaje natural de algún idioma (*source*) y traducirlo a otro texto en lenguaje natural de otro idioma (*target*), donde cada texto tendrá un largo variable. Lo cual se empleará a través de extraer información del texto *source* (*encoder*) para luego generar el texto *target* (*decoder*) en base a esta información extraída.


Deberá seleccionar el *dataset* que guste para trabajar con la tarea de traducción, comente sobre su decisión. Luego cárgelo con *pandas*.

In [0]:
import pandas as pd
import numpy as np
import tensorflow as tf
import string
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.utils import to_categorical
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Embedding, GRU, CuDNNGRU, RepeatVector, TimeDistributed, Dense, Flatten, Reshape, LSTM, Conv1D,MaxPool1D,GlobalMaxPooling1D,GlobalAveragePooling1D,BatchNormalization, InputLayer

Using TensorFlow backend.


De todos los dataset disponibles, como la finalidad es trabajar con redes neuronales, consideramos aquellos que tienen mayor cantidad de sentencias y dentro de esos, algunos en los que se trabaja con idiomas que consideramos no tan "extraños" o dentro de nuestro conocimiento.  

•	German - English deu-eng.zip (194408)  
•	French - English fra-eng.zip (168980)  
•	Portuguese - English por-eng.zip (143700)  
•	Spanish - English spa-eng.zip (121817)   

Para esta pregunta, trabajaremos con Portugues porque notamos que tenia mayor similitud de estructura que los otros lenguajes. A diferencia de el portugues, español o frances, el alemán tiene la misma raíz que el inglés pero notamos que habian muchas palabras compuestas que su correspondiente traducción en ingles era un conjunto de palabras como por ejemplo gespenstergesicht que significa cara de fantasma. 

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/por.txt", sep="\t", names=["Source","Target"])
df.head()


,Source,Target
0,Go.,Vai.
1,Go.,Vá.
2,Hi.,Oi.
3,Run!,Corre!
4,Run!,Corra!


### a) Visualice los datos ¿Qué es la entrada y qué es la salida? Comente sobre los múltiples significados/sinónimos que puede tener una palabra al ser traducida y cómo propondría arreglar eso. *se espera que pueda implementarlo*

In [0]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145693 entries, 0 to 145692
Data columns (total 2 columns):
Source    145693 non-null object
Target    145693 non-null object
dtypes: object(2)
memory usage: 2.2+ MB


In [0]:
df

,Source,Target
0,Go.,Vai.
1,Go.,Vá.
2,Hi.,Oi.
3,Run!,Corre!
4,Run!,Corra!
5,Run!,Corram!
6,Run.,Corre!
7,Run.,Corra!
8,Run.,Corram!
9,Who?,Quem?


La entrada (Soruce en el DataFrame) corresponde a una palabra u oración en inglés y la salida (Target en el DataFrame) corresponde a la palabra u oración traducida al portugues.  

In [0]:
df[0:100]

,Source,Target
0,Go.,Vai.
1,Go.,Vá.
2,Hi.,Oi.
3,Run!,Corre!
4,Run!,Corra!
5,Run!,Corram!
6,Run.,Corre!
7,Run.,Corra!
8,Run.,Corram!
9,Who?,Quem?


In [0]:
print('Para la palabra Run! :')
for i in range(135671):
    if df['Source'][i]== 'Run!':
        print(df['Target'][i])

print('\nPara la palabra Hello! :')
for i in range(135671):
    if df['Source'][i]== 'Hello!':
        print(df['Target'][i])
        
print('\nPara la palabra Wow! :')
for i in range(135671 ):
    if df['Source'][i]== 'Wow!':
        print(df['Target'][i])

Para la palabra Run! :
Corre!
Corra!
Corram!

Para la palabra Hello! :
Oi.
Alô.
Olá!
Alô!

Para la palabra Wow! :
Uau!
Nossa!
Wow!


En los ejemplos anteriores se ve que para una misma palabra en inglés, existen diferentes traducciones al portugues. En el caso de  $Run!$, de $Hello!$ y de $Wow!$ no se observa un problema muy grande debido a que los outputs correspondientes vienen de una misma raiz y solo cambian las conjugaciones y tiempos verbales. Por ende, para estos casos se propone mantener solo la raíz de las palabras.  
En los ejemplos que revisamos anteriormente no encontramos palabras que tengan múltiples significados. Sin embargo, puede que existan casos en los que si ocurra. Si la ocurrencia de esto fuera alta, se debería deducir el significado válido de la palabra según el contexto de la oración en la que aparece.

### b) Realice un pre-procesamiento a los textos como se acostumbra para eliminar símbolos inecesarios u otras cosas que estime conveniente, comente sobre la importancia de éste paso. Además de ésto deberá agregar un símbolo al final de la sentencia *target* para indicar un "alto" cuando la red neuronal necesite aprender a generar una sentencia.

In [0]:
table = str.maketrans('', '', string.punctuation) 
def clean_text(text, where=None):
    """ OJO: Sin eliminar el significado de las palabras."""
    text = text.lower()
    tokenize_text = text.split()
    tokenize_text = [word.translate(table) for word in tokenize_text]#eliminar puntuacion
    tokenize_text = [word for word in tokenize_text if word.isalpha()] #remove numbers
    if where =="target":
        tokenize_text = tokenize_text + ["#end"]  #Se agrega el símbolo que indica el "alto" para que la red pueda aprender la sentencia
    return tokenize_text
texts_input = list(df['Source'].apply(clean_text))
texts_output = list(df['Target'].apply(clean_text, where='target'))

Se muestran los resultados del preprocesamiento del texto, para los input y los output:

In [0]:
texts_input

[['go'],
 ['go'],
 ['hi'],
 ['run'],
 ['run'],
 ['run'],
 ['run'],
 ['run'],
 ['run'],
 ['who'],
 ['wow'],
 ['wow'],
 ['wow'],
 ['fire'],
 ['help'],
 ['help'],
 ['jump'],
 ['jump'],
 ['jump'],
 ['stop'],
 ['stop'],
 ['wait'],
 ['wait'],
 ['wait'],
 ['go', 'on'],
 ['hello'],
 ['hello'],
 ['hello'],
 ['hello'],
 ['i', 'ran'],
 ['i', 'see'],
 ['i', 'try'],
 ['i', 'try'],
 ['i', 'won'],
 ['i', 'won'],
 ['oh', 'no'],
 ['relax'],
 ['relax'],
 ['shoot'],
 ['smile'],
 ['smile'],
 ['attack'],
 ['attack'],
 ['attack'],
 ['cheers'],
 ['freeze'],
 ['get', 'up'],
 ['get', 'up'],
 ['get', 'up'],
 ['get', 'up'],
 ['get', 'up'],
 ['go', 'now'],
 ['got', 'it'],
 ['got', 'it'],
 ['got', 'it'],
 ['got', 'it'],
 ['he', 'ran'],
 ['he', 'ran'],
 ['hop', 'in'],
 ['hop', 'in'],
 ['hug', 'me'],
 ['i', 'fell'],
 ['i', 'know'],
 ['i', 'know'],
 ['i', 'left'],
 ['i', 'paid'],
 ['i', 'quit'],
 ['i', 'work'],
 ['im', 'ok'],
 ['im', 'ok'],
 ['im', 'up'],
 ['listen'],
 ['listen'],
 ['listen'],
 ['listen'],
 ['listen'

In [0]:
texts_output

[['vai', '#end'],
 ['vá', '#end'],
 ['oi', '#end'],
 ['corre', '#end'],
 ['corra', '#end'],
 ['corram', '#end'],
 ['corre', '#end'],
 ['corra', '#end'],
 ['corram', '#end'],
 ['quem', '#end'],
 ['uau', '#end'],
 ['nossa', '#end'],
 ['wow', '#end'],
 ['fogo', '#end'],
 ['ajuda', '#end'],
 ['socorro', '#end'],
 ['pule', '#end'],
 ['pulem', '#end'],
 ['pule', '#end'],
 ['pare', '#end'],
 ['parem', '#end'],
 ['espere', '#end'],
 ['espere', '#end'],
 ['esperem', '#end'],
 ['vá', '#end'],
 ['oi', '#end'],
 ['alô', '#end'],
 ['olá', '#end'],
 ['alô', '#end'],
 ['eu', 'corri', '#end'],
 ['estou', 'vendo', '#end'],
 ['eu', 'tento', '#end'],
 ['tento', '#end'],
 ['ganhei', '#end'],
 ['eu', 'venci', '#end'],
 ['ah', 'não', '#end'],
 ['relaxe', '#end'],
 ['relaxa', '#end'],
 ['tiro', '#end'],
 ['sorria', '#end'],
 ['sorriam', '#end'],
 ['atacar', '#end'],
 ['ataquem', '#end'],
 ['ataque', '#end'],
 ['saúde', '#end'],
 ['parado', '#end'],
 ['levantese', '#end'],
 ['levantemse', '#end'],
 ['levantat

La importancia de este paso es muy alta ya que un símbolo de diferencia al inicio o al final de una palabra podria hacer que la red neuronal reconozca a como dos palabras diferentes a una misma palabra acompañada o no por un símbolo  (como por ejemplo en español ""¡Hola!"" con "Hola", o en inglés "How are you?" con "How are you").  
Por otra parte, el símbolo de término (#end en este caso) debe ser un string que no esté presente en las palabras del texto para no producir errores.

### Cree un conjunto de validación y de pruebas fijos de $N_{exp} = 10000$ datos ¿Cuántos datos quedan para entrenar? 

In [0]:
N_exp=10000
X_train_l, X_test_l, Y_train_l, Y_test_l = train_test_split(texts_input, texts_output, test_size=N_exp, random_state=22)
X_train_l, X_val_l, Y_train_l, Y_val_l = train_test_split(X_train_l, Y_train_l,test_size=N_exp, random_state=22)

In [0]:
print('X_train_l tiene', len(X_train_l), 'elementos.')
print('Y_train_l tiene', len(Y_train_l), 'elementos.')
print('X_val_l tiene', len(X_val_l), 'elementos.')
print('Y_val_l tiene', len(Y_val_l), 'elementos.')
print('X_test_l tiene', len(X_test_l), 'elementos.')
print('Y_test_l tiene', len(Y_test_l), 'elementos.')

X_train_l tiene 125693 elementos.
Y_train_l tiene 125693 elementos.
X_val_l tiene 10000 elementos.
Y_val_l tiene 10000 elementos.
X_test_l tiene 10000 elementos.
Y_test_l tiene 10000 elementos.


Debido a que el idioma que se eligió (portugues) contiene muchas sentencias, la cantidad de datos disponibles para entrenar es de $115.671$ ejemplos.

### c) Genere un vocabulario, **desde el conjunto de entrenamiento**, sobre las palabras a recibir y generar en la traducción, esto es codificarlas a un valor entero que servirá para que la red las vea en una representación útil a procesar, *comience desde el 1 debido a que el cero será utilizado más adelante*. Para reducir el vocabulario considere las palabras que aparecen un mínimo de *min_count* veces en todo los datos, se aconseja un valor de 3. Comente sobre la importancia de ésto al reducir el vocabulario ¿De qué tamaño es el vocabulario de entrada y salida? ¿La diferencia de ésto podría ser un factor importante?

In [0]:
def create_vocab(texts, min_count=1):
    count_vocab = {}
    for sentence in texts:
        for word in sentence:
            if word not in count_vocab:
                count_vocab[word] = 1
            else:
                count_vocab[word] += 1
    return [word for word,count in count_vocab.items() if count >= min_count]

In [0]:
vocab_source = create_vocab(X_train_l, min_count=3)
word2idx_s = {w: i+1 for i, w in enumerate(vocab_source)} #index (i+1) start from 1,2,3,...
idx2word_s = {i+1: w for i, w in enumerate(vocab_source)}
n_words_s = len(vocab_source)

vocab_target = create_vocab(Y_train_l, min_count=3)
word2idx_t = {w: i+1 for i, w in enumerate(vocab_target)} #Converting text to numbers
idx2word_t = {i+1: w for i, w in enumerate(vocab_target)}#Converting number to text
n_words_t = len(vocab_target)

In [0]:
print("El largo del vocabulario source es de",n_words_s,"palabras.")
print("El largo del vocabulario target es de",n_words_t,"palabras.")
print("Por lo que la diferencia es de",str(int(n_words_t)-int(n_words_s)), "palabras.")

En el preprocesamiento, es muy importante reducir la cantidad de palabras y no considerar las que salen pocas veces, con esto se ayuda a que el modelo evite el overfitting que pueda existir al aprender palabras con muy baja ocurrencia "de memoria".    

La diferencia del tamaño de los vocabularios es, efectivamente, un factor a considerar porque si la cantidad de palabras del vocabulario inicial es más grande, puede que muchas palabras del vocabulario de entrada tengan como significado una palabra del vocabulario de salida. En el caso contrario, podria resultar perjudicial para el aprendizaje que una sola palabra en el vocabulario de inicio tenga múltiples significados o correspondencias en el vocabulario de salida.    

En particular, en nuestro caso, la diferencia es de $2.806$ palabras; un valor alto para las cantidades de palabras que estamos tratando en nuestros vocabularios, con una mayor cantidad de palabras en el vocabulario de salida.  Cabe señalar que el método de aprendizaje a considerar no traducirá palabra a palabra si no que considerará, mediante herramientas de recurrencia, el contexto de una oración. De ahí  que el problema que se mencionó anteriormente, deja de influir tanto en el aprendizaje.  (No se trata como un problema de clases desbalanceadas)

### Ahora codifique las palabras a los números indexados con el vocabulario. Recuerde que si una palabra en los otros conjuntos, o en el mismo de entrenamiento, no aparece en el vocabulario no se podrá generar una codificación, por lo que será **ignorada** ¿Cómo se podría evitar ésto?

In [0]:
""" Source/input data """
dataX_train = [[word2idx_s[word] for word in sent if word in word2idx_s] for sent in X_train_l]
dataX_valid = [[word2idx_s[word] for word in sent if word in word2idx_s] for sent in X_val_l]
dataX_test = [[word2idx_s[word] for word in sent if word in word2idx_s] for sent in X_test_l]

""" Target/output data """
dataY_train = [[word2idx_t[word] for word in sent if word in word2idx_t] for sent in Y_train_l]
dataY_valid = [[word2idx_t[word] for word in sent if word in word2idx_t] for sent in Y_val_l] 
dataY_test = [[word2idx_t[word] for word in sent if word in word2idx_t] for sent in Y_test_l] 


In [0]:
dataX_train

[[1, 2, 3, 4, 5, 6, 7, 8, 9],
 [10, 11, 12],
 [13, 14, 15, 16],
 [17, 18, 5, 19, 20, 21],
 [22, 23, 24, 25, 26, 27, 28, 29, 30, 31],
 [32, 14, 33, 34, 35, 36],
 [37, 38, 28, 39, 35, 40, 41],
 [42, 43, 44, 45],
 [46, 47],
 [48, 49, 50, 51],
 [52, 2, 53, 54],
 [55, 40, 56],
 [57, 58, 59],
 [48, 60, 50, 35, 40, 61, 62],
 [10, 63, 64, 65],
 [66, 67, 51, 68, 69, 70, 14, 71],
 [72, 14, 73, 74, 75, 76, 77],
 [42, 2, 40, 26, 78, 79],
 [17, 80, 18, 81, 23, 14, 82, 42, 83, 84],
 [17, 85, 86, 87, 10],
 [88, 67, 5, 89, 90, 91, 92],
 [17, 18, 5, 93, 94],
 [70, 95, 96, 76, 97, 98],
 [17, 63, 29, 99, 100],
 [93, 101, 102, 10, 103, 40, 65, 104],
 [105, 10, 19, 106, 107],
 [17, 108, 109, 14, 110],
 [111, 2, 58, 112, 113, 114],
 [115, 110, 2, 116, 117, 35, 84],
 [42, 118, 14, 119, 120, 121],
 [37, 122, 123, 124, 23, 125, 126],
 [42, 127, 128, 23, 10],
 [129, 130, 131],
 [17, 18, 5, 132, 133],
 [14, 134, 135, 136, 137],
 [138, 139, 58, 140, 141, 17, 142, 1],
 [105, 10, 122, 143],
 [14, 144, 145, 146, 147

Para evitar que una palabra poco frecuente, que no posea una etiqueta se aparezca, la solución que se nos ocurre es "limpiar" nuestro dataset al momento de rear el vocabulario, eliminando los input (palabras ofrases) que contienen palabras con una ocurrencia menor al número elegido para construir el vocabulario, en este caso, 3. Así, todas las oraciones disponibles en el dataset no contendrían palabras ignoradas.   

Para lo que sigue, no se realizará la estrategia propuesta debido al costo computacional que requiere (recorrer secuencialmente el dataset que es de gran longitud varias veces)  y los buenos resultados obtenidos a continuación.  

###  d) Debido al largo variable de los textos de entrada y salida será necesario estandarizar ésto para poder trabajar de manera más cómoda en Keras, *cada texto (entrada y salida) pueden tener distinto largo máximo*. Comente sobre la decisión del tipo de *padding*, *pre o post* ¿Qué sucede al variar el largo máximo de instantes de tiempo para procesar en cada parte del modelo (entrada y salida)?

In [0]:
""" INPUT DATA (Origin language) """
max_inp_length = max(map(len,dataX_train))
print("Largo max inp: ",max_inp_length)
word2idx_s["*"] = 0 #padding symbol
idx2word_s[0] = "*"
n_words_s += 1  
X_train = sequence.pad_sequences(dataX_train, maxlen=max_inp_length, padding='pre', value=word2idx_s["*"])
X_val = sequence.pad_sequences(dataX_valid, maxlen=max_inp_length, padding='pre', value=word2idx_s["*"])
X_test = sequence.pad_sequences(dataX_test, maxlen=max_inp_length, padding='pre', value=word2idx_s["*"])


""" OUTPUT DATA (Destination language) """
max_out_length = max(map(len,dataY_train)) 
print("Largo max out: ",max_out_length)
word2idx_t["*"] = 0 #padding symbol
idx2word_t[0] = "*"
n_words_t += 1  
Y_train = sequence.pad_sequences(dataY_train, maxlen=max_out_length, padding='post', value=word2idx_t["*"])
Y_val = sequence.pad_sequences(dataY_valid, maxlen=max_out_length, padding='post', value=word2idx_t["*"])
Y_test = sequence.pad_sequences(dataY_test, maxlen=max_out_length, padding='post', value=word2idx_t["*"])


Largo max inp:  35
Largo max out:  34


In [0]:
X_train

array([[   0,    0,    0, ...,    7,    8,    9],
       [   0,    0,    0, ...,   10,   11,   12],
       [   0,    0,    0, ...,   14,   15,   16],
       ...,
       [   0,    0,    0, ...,  351,   35,   10],
       [   0,    0,    0, ...,   89,  245,   89],
       [   0,    0,    0, ...,  285, 1058,  315]], dtype=int32)

In [0]:
Y_train

array([[   1,    2,    3, ...,    0,    0,    0],
       [  11,   12,   13, ...,    0,    0,    0],
       [  15,    3,   16, ...,    0,    0,    0],
       ...,
       [ 335,   13,   55, ...,    0,    0,    0],
       [ 150,   11, 1012, ...,    0,    0,    0],
       [  76,  223,  701, ...,    0,    0,    0]], dtype=int32)

El tipo de Padding realizado es de "pre" en el conjunto de input y "post"  en el conjunto de output, esta decisión se debe a la estuctura de la red neuronal con la que se trabajará. Como se trabajará con Encoder-Decoder,  es recomendable que las paabras "útiles" (las que no son 0 en el) estén juntas, lo que se logra con el padding anterioremente de descrito.

El largo máximo de instantes de tiempo para procesar debe ser fijo, ya que si fuese variable fallarán las conexiones entre las capas al procesar la red neuronal.

###  e) Para evitar que la red obtenga una ganancia por imitar/predecir el símbolo de *padding* que está bastante presente en los datos coloque un peso sobre éste clase, con valor 0, así se evita que tenga impacto en la función objetivo. Ya que *keras* no soporta directamente ésto en series de tiempo coloque el peso a cada instante de tiempo de cada dato de entrenamiento dependiendo de su clase. Comente sobre alguna otra forma en que se podría manejar el evitar que la red prediga en mayoría el símbolo de *padding*.

In [0]:
c_weights = np.ones(n_words_t)
c_weights[0] = 0 #padding class masked
sample_weight = np.zeros(Y_train.shape)
for i in range(sample_weight.shape[0]):
    sample_weight[i] = c_weights[Y_train[i,:]]
#Crea la matriz de pesos asociada a Y_train

A continuación se ve un ejemplo donde, efectivamente, el peso del símbolo de padding es 0 y el de las demás palabras codificadas es 1.

In [0]:
Y_train[9]

array([50,  6, 51, 10,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
      dtype=int32)

In [0]:
sample_weight[9]

array([1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

Otra forma en que se nos ocurre que se podria evitar la predicción del símbolo de padding por parte de la red podria ser considerar una función de pérdida que este construida para "evitar" de cierta forma que los símbolos codificados con 0 (que corresponden al padding) mejoren el entrenamiento. Esta función podría construirse a mano y el resultado debiese ser similar a la estrategia de los pesos que se propone.

### f) Para lograr la tarea defina una red recurrente del tipo *encoder*-*decoder* como la que se presenta en la siguiente imágen. <img src="https://chunml.github.io/ChunML.github.io/images/projects/sequence-to-sequence/repeated_vector.png" width="60%" />
###En primer lugar defina el *Encoder* que procesara el texto de entrada y retornará un solo vector final, haciendo uso de las capas ya conocidas de *Embedding* para generar un vector denso de palabra y *GRU*, pero en su versión acelerada para GPU.

### Luego defina la sección que conecta el largo (*timesteps*) de entrada *vs* el de salida.

### Finalmente defina el *Decoder* para generar la secuencia de salida en texto de palabras en otro idioma, a través de la función *softmax* sobre cada instante de tiempo (*timestep*).

### Entrene la red entre 1 a 5 *epochs*, agregando los pesos definidos sobre cada ejemplo de entrenamiento. Además de utilizar una función de pérdida que evita generar explícitamente los *one hot vector*

Se define el Encoder formado por una capa de Embedding y dos capas de GRU (Gated Recurrent Unit), la sección que conecta el Encoder con el Decoder ser forma por una capa de RepeatVector, que repite el input de dicha capa tantas veces como el largo de salida.Luego, se define el Decoder se forma de dos capas de GRU y una capa de salida TimeDistributed  de capa Densa (se le aplica una capa densa a cada segmento temporal del input con función de activación softmax). 

Se considera la función de pérdida 'sparse_categorical_crossentropy', que se porta como la ya conocida 'categorical_crossentropy' pero permite trabajar con salidas que estan codificadas como números enteros y no en una representación one-hot-vector. (Efectivamente es nuestro caso, dado que cada coordenada de salida corresponde a un número entero no negativo)

Finalmente, se entrena la red para 3 epochs.

In [0]:
Y_train=Y_train[:,:,None]
Y_val=Y_val[:,:,None]
Y_test=Y_test[:,:,None]

In [0]:

EMBEDDING_DIM = 100
model = Sequential()
model.add(Embedding(input_dim=n_words_s, output_dim=EMBEDDING_DIM, input_length=max_inp_length))
model.add(CuDNNGRU(64, return_sequences=True))
model.add(CuDNNGRU(128, return_sequences=False))
model.add(RepeatVector(max_out_length))
model.add(CuDNNGRU(128, return_sequences=True))
model.add(CuDNNGRU(64, return_sequences=True))
model.add(TimeDistributed(Dense(n_words_t,activation='softmax')))
model.summary()
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', sample_weight_mode='temporal')
model.fit(X_train, Y_train, epochs=3, batch_size=256, validation_data=(X_val, Y_val), sample_weight = sample_weight) 

W0804 15:09:33.448425 140589869713280 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0804 15:09:33.474615 140589869713280 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0804 15:09:33.477839 140589869713280 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0804 15:09:35.037471 140589869713280 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0804 15:09:35.062474 140589869713280 deprecation_wrappe

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 35, 100)           616200    
_________________________________________________________________
cu_dnngru_1 (CuDNNGRU)       (None, 35, 64)            31872     
_________________________________________________________________
cu_dnngru_2 (CuDNNGRU)       (None, 128)               74496     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 34, 128)           0         
_________________________________________________________________
cu_dnngru_3 (CuDNNGRU)       (None, 34, 128)           99072     
_________________________________________________________________
cu_dnngru_4 (CuDNNGRU)       (None, 34, 64)            37248     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 34, 9107)          591955    
Total para

W0804 15:09:35.654662 140589869713280 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 125693 samples, validate on 10000 samples
Epoch 1/3
125693/125693 [==============================] - 55s 439us/step - loss: 5.9784 - val_loss: 12.5771
Epoch 2/3
125693/125693 [==============================] - 50s 395us/step - loss: 5.7710 - val_loss: 13.4363
Epoch 3/3
125693/125693 [==============================] - 49s 392us/step - loss: 5.7187 - val_loss: 13.9702


Se observa una disminución de la función de pérdida durante el paso de las epochs.

### g) Debido a lo costoso de tener una red completamente recurrente para entrenar y poder experimentar, cambie el modelo que procesa el *Encoder* por una red convolucional, reduciendo el número de capas pero aumentando las neuronas. Utilice tamaños de *kernel*  igual a 5 y funciones de activaciones relu. Se agregan capas de *BatchNormalization* debido a que en el *Decoder* contamos con redes recurrentes que tienen capa activación distinta a la usada por las convoluciones. La capa de *GlobalMaxPooling1d* es lo que permite reducir toda la información extraída a un único vector, como se realizó anteriormente con *return_sequences=False*, comente sobre la ganancia o desventaja de ésto *vs* la red neuronal.

In [0]:
model1 = Sequential()
model1.add(Embedding(input_dim=n_words_s, output_dim=EMBEDDING_DIM, input_length=max_inp_length))
model1.add(Conv1D(256, 5, padding='same', activation='relu', strides=1))
model1.add(BatchNormalization()) #for stability
model1.add(Conv1D(256, 5, padding='same', activation='relu', strides=1))
model1.add(BatchNormalization())
model1.add(GlobalMaxPooling1D()) #aka to return_sequences=False
model1.add(RepeatVector(max_out_length)) #conection
model1.add(CuDNNGRU(256, return_sequences=True))
model1.add(TimeDistributed(Dense(n_words_t, activation='softmax')))
model1.summary() 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 35, 100)           616200    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 35, 256)           128256    
_________________________________________________________________
batch_normalization_1 (Batch (None, 35, 256)           1024      
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 35, 256)           327936    
_________________________________________________________________
batch_normalization_2 (Batch (None, 35, 256)           1024      
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 256)               0         
_________________________________________________________________
repeat_vector_2 (RepeatVecto (None, 34, 256)           0         
__________

### Entrene el modelo igual a lo presentado anteriormente pero ahora por 20 *epochs* ¿Cambian los tiempos de procesamiento y la cantidad de parámetros?

In [0]:
model1.compile(optimizer='adam', loss='sparse_categorical_crossentropy', sample_weight_mode='temporal')

In [0]:
model1.fit(X_train, Y_train, epochs=20, batch_size=256,validation_data=(X_val, Y_val), sample_weight = sample_weight) 

Train on 125693 samples, validate on 10000 samples
Epoch 1/20
125693/125693 [==============================] - 64s 512us/step - loss: 5.0818 - val_loss: 13.6940
Epoch 2/20
125693/125693 [==============================] - 62s 495us/step - loss: 3.8197 - val_loss: 13.5998
Epoch 3/20
125693/125693 [==============================] - 62s 497us/step - loss: 3.2029 - val_loss: 13.5234
Epoch 4/20
125693/125693 [==============================] - 62s 497us/step - loss: 2.8118 - val_loss: 13.4666
Epoch 5/20
125693/125693 [==============================] - 63s 498us/step - loss: 2.5392 - val_loss: 13.4342
Epoch 6/20
125693/125693 [==============================] - 63s 499us/step - loss: 2.3287 - val_loss: 13.4138
Epoch 7/20
125693/125693 [==============================] - 62s 497us/step - loss: 2.1763 - val_loss: 13.4002
Epoch 8/20
125693/125693 [==============================] - 62s 496us/step - loss: 2.0440 - val_loss: 13.3870
Epoch 9/20
125693/125693 [==============================] - 62s 496us

Para el primero modelo se tenian 1.407.653 parámetros entrenables y para el segundo modelo 3.699.045 parámetros entrenables (2,5 veces la cantidad de parámetros del primer modelo) y aún así, los tiempos de entrenamiento de cada epoch son similares.  Esto último se debe a la disminución de las capas recurrentes en el segundo modelo respecto al primero.

### h) Visualice lo aprendido por el modelo sobre algunos datos del conjunto de entrenamiento y validación, comente lo observado.

In [0]:
def predict_words(y_indexs, data="target"):
    """ Predict until '-#end-' is seen """
    return_val = []
    for indx_word in y_indexs:
        if indx_word != 0: #start to predict
            return_val.append(np.squeeze(indx_word))
            if data == "target": #if target is predicting
                if indx_word == word2idx_t["#end"]:
                    return return_val                
    return return_val


Visualizamos lo aprendido par el primer modelo:

In [0]:
n_s = 50
Y_set=Y_train
X_set=X_train
idx = np.random.choice(np.arange(Y_set.shape[0]), size=n_s, replace=False)
Y_set_pred = model.predict_classes(X_set[idx] )
for i, n_sampled in enumerate(idx):
    text_input = [idx2word_s[p] for p in predict_words(X_set[n_sampled], data="source")]
    print("Texto source: ", ' '.join(text_input))
    text_real = [idx2word_t[p] for p in predict_words(Y_set[n_sampled,:,0], data="target")]
    print("Texto target real: ", ' '.join( text_real))
    text_sampled = [idx2word_t[p] for p in predict_words(Y_set_pred[i], data="target")]
    print("Texto target predicho del modelo 0: ", ' '.join(text_sampled))

Texto source:  the price is low but the quality isnt very good
Texto target real:  o preço é baixo mas a qualidade não é muito alta #end
Texto target predicho del modelo 0:  #end
Texto source:  owls hunt at night
Texto target real:  as corujas caçam à noite #end
Texto target predicho del modelo 0:  #end
Texto source:  i dedicate this song to you
Texto target real:  dedico esta música a você #end
Texto target predicho del modelo 0:  #end
Texto source:  i can only pay dollars at most
Texto target real:  eu posso pagar dólares no máximo #end
Texto target predicho del modelo 0:  #end
Texto source:  please close the door
Texto target real:  fechem a porta por favor #end
Texto target predicho del modelo 0:  #end
Texto source:  tom pointed toward the ceiling
Texto target real:  tom apontou em direção ao teto #end
Texto target predicho del modelo 0:  #end
Texto source:  youre way taller than tom
Texto target real:  você é mais alta que o tom #end
Texto target predicho del modelo 0:  #end
Texto

In [0]:
n_s = 50
Y_set=Y_val
X_set=X_val
idx = np.random.choice(np.arange(Y_set.shape[0]), size=n_s, replace=False)
Y_set_pred = model.predict_classes(X_set[idx] )
for i, n_sampled in enumerate(idx):
    text_input = [idx2word_s[p] for p in predict_words(X_set[n_sampled], data="source")]
    print("Texto source: ", ' '.join(text_input))
    text_real = [idx2word_t[p] for p in predict_words(Y_set[n_sampled,:,0], data="target")]
    print("Texto target real: ", ' '.join( text_real))
    text_sampled = [idx2word_t[p] for p in predict_words(Y_set_pred[i], data="target")]
    print("Texto target predicho del modelo 0: ", ' '.join(text_sampled))

Texto source:  tell tom im hungry
Texto target real:  diga ao tom que estou com fome #end
Texto target predicho del modelo 0:  #end
Texto source:  youre cute
Texto target real:  vocês são bonitos #end
Texto target predicho del modelo 0:  #end
Texto source:  id like to see your father
Texto target real:  gostaria de ver o teu pai #end
Texto target predicho del modelo 0:  #end
Texto source:  didnt you use to have a car
Texto target real:  a senhora não tinha um carro #end
Texto target predicho del modelo 0:  #end
Texto source:  it doesnt matter anymore
Texto target real:  não importa mais #end
Texto target predicho del modelo 0:  #end
Texto source:  is tom lost
Texto target real:  o tom está perdido #end
Texto target predicho del modelo 0:  #end
Texto source:  what are they doing
Texto target real:  o que eles estão fazendo #end
Texto target predicho del modelo 0:  #end
Texto source:  i knew that it wouldnt be easy
Texto target real:  eu sabia que não seria fácil #end
Texto target predic

Visualizamos lo aprendido para el segundo modelo:

In [0]:
n_s = 50
Y_set=Y_train
X_set=X_train
idx = np.random.choice(np.arange(Y_set.shape[0]), size=n_s, replace=False)
Y_set_pred = model1.predict_classes(X_set[idx] )
for i, n_sampled in enumerate(idx):
    text_input = [idx2word_s[p] for p in predict_words(X_set[n_sampled], data="source")]
    print("Texto source: ", ' '.join(text_input))
    text_real = [idx2word_t[p] for p in predict_words(Y_set[n_sampled,:,0], data="target")]
    print("Texto target real: ", ' '.join( text_real))
    text_sampled = [idx2word_t[p] for p in predict_words(Y_set_pred[i], data="target")]
    print("Texto target predicho del modelo 1: ", ' '.join(text_sampled))

Texto source:  tom learned to skate when he was thirteen
Texto target real:  tom aprendeu a patinar quando tinha treze anos #end
Texto target predicho del modelo 1:  tom aprendeu a a quando quando treze treze #end
Texto source:  everyone should know that
Texto target real:  todos devem saber disso #end
Texto target predicho del modelo 1:  todos devem saber disso #end
Texto source:  who are you im tom
Texto target real:  quem é você sou o tom #end
Texto target predicho del modelo 1:  quem você você eu sou tom #end
Texto source:  im sure that your parents are very proud of you
Texto target real:  tenho certeza de que seus pais têm muito orgulho de vocês #end
Texto target predicho del modelo 1:  tenho certeza de que que de orgulhosos de de de de de #end
Texto source:  give this copy to tom
Texto target real:  dê esta cópia ao tom #end
Texto target predicho del modelo 1:  dê a cópia ao tom tom #end
Texto source:  its making me nervous
Texto target real:  está me deixando nervoso #end
Texto

In [0]:
n_s = 50
Y_set=Y_val
X_set=X_val
idx = np.random.choice(np.arange(Y_set.shape[0]), size=n_s, replace=False)
Y_set_pred = model1.predict_classes(X_set[idx] )
for i, n_sampled in enumerate(idx):
    text_input = [idx2word_s[p] for p in predict_words(X_set[n_sampled], data="source")]
    print("Texto source: ", ' '.join(text_input))
    text_real = [idx2word_t[p] for p in predict_words(Y_set[n_sampled,:,0], data="target")]
    print("Texto target real: ", ' '.join( text_real))
    text_sampled = [idx2word_t[p] for p in predict_words(Y_set_pred[i], data="target")]
    print("Texto target predicho del modelo 1: ", ' '.join(text_sampled))

Texto source:  i dont feel like joking
Texto target real:  eu não estou com vontade de brincar #end
Texto target predicho del modelo 1:  eu não estou com de de de #end
Texto source:  im starting to get bored
Texto target real:  estou começando a ficar entediado #end
Texto target predicho del modelo 1:  estou começando a estudar ficar #end
Texto source:  you should be more of your safety
Texto target real:  você deveria estar mais preocupado com sua segurança #end
Texto target predicho del modelo 1:  você deveria ser mais mais de sua sua #end
Texto source:  dont worry well find tom
Texto target real:  não se preocupe nós vamos achar o tom #end
Texto target predicho del modelo 1:  não se preocupe com encontrar o o #end
Texto source:  she divided the cake between the two
Texto target real:  ela repartiu o bolo entre os dois #end
Texto target predicho del modelo 1:  ela dividiu o bolo entre entre dois #end
Texto source:  tom sat down on a rock
Texto target real:  tom sentouse em uma rocha 

Como se esperaba al comparar los valores de las funciones de pérdida, el segundo modelo (no puramente recurrente)  tiene un rendimiento mucho mejor tanto en el conjunto de entrenamiendo como en el de validación.

### i) Realice algún cambio esperando que mejore el modelo entrenado, luego vuelva a visualizar lo predicho por la red *vs* lo real. *Debido a lo costoso en entrenar puede optar por realizar solo un cambio pero que sea significativo*.  Se comentan algunas opciones para utilizar y combinar:
* Cambiar  el *embedding* por alguno pre-entrenado
* Agregar regularizadores
* Asignar peso a las clases/palabras de salida
* Cambiar *Global max pooling* por *Average max pooling*
* Aumentar o reducir capas
* Aumentar o reducir neuronas/unidades  

Para mejorar el rendimiento del modelo, tomamos el segundo modelo de los dos anteriores y modificamos Global Max pooling por Global Average pooling.

In [0]:
model2 = Sequential()
model2.add(Embedding(input_dim=n_words_s, output_dim=EMBEDDING_DIM, input_length=max_inp_length))
model2.add(Conv1D(256, 5, padding='same', activation='relu', strides=1))
model2.add(BatchNormalization()) #for stability
model2.add(Conv1D(256, 5, padding='same', activation='relu', strides=1))
model2.add(BatchNormalization())
model2.add(GlobalAveragePooling1D()) #aka to return_sequences=False
model2.add(RepeatVector(max_out_length)) #conection
model2.add(CuDNNGRU(256, return_sequences=True))
model2.add(TimeDistributed(Dense(n_words_t, activation='softmax')))
model2.summary() 
model2.compile(optimizer='adam', loss='sparse_categorical_crossentropy', sample_weight_mode='temporal')
model2.fit(X_train, Y_train, epochs=20, batch_size=256, validation_data=(X_val, Y_val), sample_weight = sample_weight)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 35, 100)           616200    
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 35, 256)           128256    
_________________________________________________________________
batch_normalization_7 (Batch (None, 35, 256)           1024      
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 35, 256)           327936    
_________________________________________________________________
batch_normalization_8 (Batch (None, 35, 256)           1024      
_________________________________________________________________
global_average_pooling1d_3 ( (None, 256)               0         
_________________________________________________________________
repeat_vector_6 (RepeatVecto (None, 34, 256)           0         
__________

In [0]:
n_s = 50
Y_set=Y_train
X_set=X_train
idx = np.random.choice(np.arange(Y_set.shape[0]), size=n_s, replace=False)
Y_set_pred = model2.predict_classes(X_set[idx] )
for i, n_sampled in enumerate(idx):
    text_input = [idx2word_s[p] for p in predict_words(X_set[n_sampled], data="source")]
    print("Texto source: ", ' '.join(text_input))
    text_real = [idx2word_t[p] for p in predict_words(Y_set[n_sampled,:,0], data="target")]
    print("Texto target real: ", ' '.join( text_real))
    text_sampled = [idx2word_t[p] for p in predict_words(Y_set_pred[i], data="target")]
    print("Texto target predicho: ", ' '.join(text_sampled))

Texto source:  tom needs to do something
Texto target real:  tom precisa fazer alguma coisa #end
Texto target predicho:  tom precisa precisa algo algo coisa #end
Texto source:  ive given up eating meat
Texto target real:  eu parei de comer carne #end
Texto target predicho:  eu parei de comer comer #end
Texto source:  tom needs water
Texto target real:  tom precisa de água #end
Texto target predicho:  tom precisa de de água #end
Texto source:  does it snow in boston
Texto target real:  está nevando em boston #end
Texto target predicho:  está nevando em em boston #end
Texto source:  tom is not going to do it
Texto target real:  tom não vai fazer isso #end
Texto target predicho:  tom não vai vai isso isso #end
Texto source:  how are you going to pay me
Texto target real:  como você vai me pagar #end
Texto target predicho:  você vai você vai me #end
Texto source:  tom seems to be a bit taller than mary
Texto target real:  tom parece ser um pouco mais alto do que mary #end
Texto target pred

Notamos que el modelo predice muchas veces "tom" y en algunos ejemplos predice "tom" en lugares que no corresponde. Esto es porque muchos ejemplos tienen "tom", es decir aparece bastante y esto hace que la palabra tenga una mayor probabilidad de aparecer. Proponemos para solucionar este problema tener un dataset sin un pronombre personal fijo como "tom" o setear los pesos de la red para que reconozca este tipo de palabras.

El modelo tiende a repetir conectores como "e" y "que" porque al ser conectores aparecen con bastante frecuencia lo que las hace mas probable de aparecer. Es por eso que el modelo que tiene la capa de GlobalMaxPooling tiende a predecir mas palabras como conectores, pues elige exactamente la de mayor valor mientras que GlobalAveragePooling tiende a predecir menos palabras como conectores.


###  j) A pesar de que la tarea de medir qué tan similar es un texto a otro ya es un área de investigación propia [[6]](#refs), usted deberá utilizar alguna métrica de desempeño para ver qué tan buena es la traducción del texto *versus* el texto real entregado. Debido a que la métrica de *Exact Matching* (EM) puede ser muy drástica, mida *f1 score* por texto además de proponer alguna otra técnica de evaluación para medir sobre el conjunto de pruebas y los otros conjuntos si estima conveniente. Puede basarse en otros trabajos como *Image captioning* o *Text summary*. 
*Hint: Debido a los problemas de memoria al realizar un forward-pass, solo seleccione un subconjunto $N_{sub}$ del conjunto de pruebas para realizar ésta evaluación, se aconseja entre 1000 y 5000.*

###  La función de *f1 score* en este extracto se calcula en base al *precision* y *recall* de que aparezca cada una de las palabras predichas dentro de las palabras reales (como si cada palabra fuera una clase de "aparece" o no), **sin importar el orden ni la ocurrencia**.

In [0]:
m = MultiLabelBinarizer().fit([np.arange(n_words_t)]) 
def calculate_f1(true, pred):
    true = np.squeeze(true)
    pred = np.squeeze(pred)
    binarized_true = m.transform([predict_words(true)])[0] #onehot of words appear
    binarized_pred = m.transform([predict_words(pred)])[0] #onehot of words appear
    return f1_score(binarized_true, binarized_pred, average='binary') #only on appearing words


Consideraremos un subconjunto del conjunto de pruebas de 2000 ejemplos.

In [0]:
n_s = 2000
Y_set=Y_test
X_set=X_test
idx = np.random.choice(np.arange(Y_set.shape[0]), size=n_s, replace=False)

In [0]:
Y_set_pred = model.predict_classes(X_set[idx] )

In [0]:
f1_final = np.mean([calculate_f1(true_words,pred_words) for true_words,pred_words in zip(Y_set[idx],Y_set_pred)])
#porcentaje
print("Para el primer modelo, f1 score es: ", f1_final*100)

Para el primer modelo, f1 score es:  28.22751078556806


In [0]:
Y_set_pred = model1.predict_classes(X_set[idx] )

In [0]:
f1_final = np.mean([calculate_f1(true_words,pred_words) for true_words,pred_words in zip(Y_set[idx],Y_set_pred)])
#porcentaje
print("Para el segundo modelo, f1 score es: ", f1_final*100)

Para el segundo modelo, f1 score es:  70.9569720619427


In [0]:
Y_set_pred = model2.predict_classes(X_set[idx] )

In [0]:
f1_final = np.mean([calculate_f1(true_words,pred_words) for true_words,pred_words in zip(Y_set[idx],Y_set_pred)])
#porcentaje
print("Para el tercer modelo, f1 score es: ", f1_final*100)

Para el tercer modelo, f1 score es:  56.498547070827996


Como se esperaba, el primer modelo presenta una f1-score mucho más baja que los demás, sin embargo, el tercer modelo presenta una f1-score más baja que la del segundo modelo pese a tener un valor final de la función de pérdida mejor (más bajo). Esta diferencia se la atribuimos a que el modelo entrena para disminuir una función de pérdida y no para aumentar un score. 

En trabajos de Image captioning usan métricas como CIDEr, METEOR, ROUGE and BLEU,.
ROUGE o Recall-Oriented Understudy for Gisting Evaluation es un paquete de metricas que evaluan la traducción automática en el procesamiento del lenguaje natural.  
BLEU o Bilingual Evaluation Understudy es una métrica de evaluación de la calidad de una traducción.

In [0]:
from nltk.translate.bleu_score import sentence_bleu

In [0]:
n_s = 2000
Y_set=Y_test
X_set=X_test
idx = np.random.choice(np.arange(Y_set.shape[0]), size=n_s, replace=False)
Y_set_pred = model.predict_classes(X_set[idx])
a=list()
b=list()
for i, n_sampled in enumerate(idx):

    text_real = [idx2word_t[p] for p in predict_words(Y_set[n_sampled,:,0], data="target")]
    a.append(text_real)
    text_sampled = [idx2word_t[p] for p in predict_words(Y_set_pred[i], data="target")]
    b.append(text_sampled)
    
suma=0
for i in range(2000):
  sumando = sentence_bleu(b[i],a[i])
  suma=suma+sumando
  
print("BLEU de modelo 1: ", suma/2000)

/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


BLEU de modelo 1:  0.018800887776109106


In [0]:
Y_set_pred = model1.predict_classes(X_set[idx])
a=list()
b=list()
for i, n_sampled in enumerate(idx):

    text_real = [idx2word_t[p] for p in predict_words(Y_set[n_sampled,:,0], data="target")]
    a.append(text_real)
    text_sampled = [idx2word_t[p] for p in predict_words(Y_set_pred[i], data="target")]
    b.append(text_sampled)
    
suma=0
for i in range(2000):
  sumando = sentence_bleu(b[i],a[i])
  suma=suma+sumando
  
print("BLEU de modelo 2: ", suma/2000)

/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


BLEU de modelo 2:  0.21366775433567084


In [0]:
Y_set_pred = model2.predict_classes(X_set[idx])
a=list()
b=list()
for i, n_sampled in enumerate(idx):

    text_real = [idx2word_t[p] for p in predict_words(Y_set[n_sampled,:,0], data="target")]
    a.append(text_real)
    text_sampled = [idx2word_t[p] for p in predict_words(Y_set_pred[i], data="target")]
    b.append(text_sampled)
    
suma=0
for i in range(2000):
  sumando = sentence_bleu(b[i],a[i])
  suma=suma+sumando
  
print("BLEU de modelo 3: ", suma/2000)

/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


BLEU de modelo 3:  0.216696293442618


Al usar la métrica BLEU (promedio) notamos una mejora (leve) en el rendimiento del tercer modelo en relación al segundo que era lo que se esperaba al cambiar la capa de GlobalMaxPooling por GlobalAveragePooling  y no se pudo apreciar en la f1-score por als razones mencionadas antes. Esta mejora en el rendimiento de BLEU se debe a que BLEU es una métrica diseñada para medir la efectividad de traducciones (que es justamente el problema que estamos tratando)

###  k) En ves de volver a variar el modelo de *Encoder*, dejaremos una representación manual explícita (*no entrenable*) a través de extraer características manuales de los textos *source*, como por ejemplo representaciones *term frequency* (TF) o TF-IDF, proporcionadas a través de __[sklearn](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.feature_extraction.text)__. Luego, con esto generado, defina y entrene el modelo *Decoder* neuronal como el presentado en las preguntas anteriores, ésto es comenzar desde la capa *RepeatVector* hasta llegar a la clasificación sobre el texto *target*. Compare el desempeño con lo presentado en (j) y lo visualizado en (h).

Term frequency **(TF)** o **TF-IDF** es  un valor numérico que representa la importancia de una palabra para un documento de una colección, el valor de esta medida va aumentando a medida que la palabra aparece más veces en el texto.  

Matemáticamente, *TF* es una medida obtenida por el producto de otras dos medidas la frecuencia de término y la frecuencia inversa del documento. La primera se refiere a las veces que aparece un término (palabra) y la segunda a la cantidad de documentos de la colección donde está presente la palabra.

In [0]:
def dummy_fun(doc):
    return doc
tf_idf = TfidfVectorizer(analyzer='word',tokenizer=dummy_fun,preprocessor=dummy_fun,
                         token_pattern=None,use_idf= True, smooth_idf=True, norm='l2')   
X_train_tfidf = tf_idf.fit_transform(dataX_train).astype('float32').todense()
X_test_tfidf = tf_idf.transform(dataX_test)
X_val_tfidf= tf_idf.transform(dataX_valid)

In [0]:
EMBEDDING_DIM = 100
model3 = Sequential()
model3.add(InputLayer( input_shape=(6161,),batch_size=256))
model3.add(RepeatVector(max_out_length)) #conection
model3.add(CuDNNGRU(256, return_sequences=True))
model3.add(TimeDistributed(Dense(n_words_t, activation='softmax')))
model3.summary() 

W0805 00:50:09.394988 139921068345216 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0805 00:50:09.436244 139921068345216 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0805 00:50:10.467472 139921068345216 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
repeat_vector_1 (RepeatVecto (256, 34, 6161)           0         
_________________________________________________________________
cu_dnngru_1 (CuDNNGRU)       (256, 34, 256)            4929792   
_________________________________________________________________
time_distributed_1 (TimeDist (256, 34, 9107)           2340499   
Total params: 7,270,291
Trainable params: 7,270,291
Non-trainable params: 0
_________________________________________________________________


In [0]:
model3.compile(optimizer='adam', loss='sparse_categorical_crossentropy', sample_weight_mode='temporal')

W0805 00:50:16.337696 139921068345216 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0805 00:50:16.365552 139921068345216 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3341: The name tf.log is deprecated. Please use tf.math.log instead.



### Por alguna razón misteriosa, la red no entrena y no queremos molestar con más mails en esta pregunta, si se ejecutan los comandos necesarios para que compile el modelo, con el código que se fdeja a continuación se puede calcular la f1 score y la BLEU promedio, para luego comparar

In [0]:
model3.fit(X_train_tfidf, Y_train, epochs=5, batch_size=256, validation_data=(X_val_tfidf, Y_val), sample_weight = sample_weight)

W0805 00:50:25.107889 139921068345216 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0805 00:50:25.257143 139921068345216 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 125693 samples, validate on 10000 samples
Epoch 1/5
125440/125693 [============================>.] - ETA: 0s - loss: 6.0743

InvalidArgumentError: ignored

In [0]:
m = MultiLabelBinarizer().fit([np.arange(n_words_t)]) 
def calculate_f1(true, pred):
    true = np.squeeze(true)
    pred = np.squeeze(pred)
    binarized_true = m.transform([predict_words(true)])[0] #onehot of words appear
    binarized_pred = m.transform([predict_words(pred)])[0] #onehot of words appear
    return f1_score(binarized_true, binarized_pred, average='binary') #only on appearing words

In [0]:
n_s = 2000
Y_set=Y_test
X_set=X_test_tfidf
idx = np.random.choice(np.arange(Y_set.shape[0]), size=n_s, replace=False)

In [0]:
Y_set_pred = model3.predict_classes(X_set[idx] )

In [0]:
f1_final = np.mean([calculate_f1(true_words,pred_words) for true_words,pred_words in zip(Y_set[idx],Y_set_pred)])
#porcentaje
print("Para el último modelo, f1 score es: ", f1_final*100)

In [0]:
from nltk.translate.bleu_score import sentence_bleu

In [0]:
n_s = 2000
Y_set=Y_test
X_set=X_test
idx = np.random.choice(np.arange(Y_set.shape[0]), size=n_s, replace=False)
Y_set_pred = model.predict_classes(X_set[idx])
a=list()
b=list()
for i, n_sampled in enumerate(idx):

    text_real = [idx2word_t[p] for p in predict_words(Y_set[n_sampled,:,0], data="target")]
    a.append(text_real)
    text_sampled = [idx2word_t[p] for p in predict_words(Y_set_pred[i], data="target")]
    b.append(text_sampled)
    
suma=0
for i in range(2000):
  sumando = sentence_bleu(b[i],a[i])
  suma=suma+sumando
  
print("BLEU del último modelo: ", suma/2000)

<a id="refs"></a>
## Referencias
[1] https://es.wikipedia.org/wiki/Ley_de_Zipf    
[2] Mikolov, T., Chen, K., Corrado, G., & Dean, J. (2013). *Efficient estimation of word representations in vector space*. arXiv preprint arXiv:1301.3781.    
[3] Cho, K., Van Merriënboer, B., Bahdanau, D., & Bengio, Y. (2014). *On the properties of neural machine translation: Encoder-decoder approaches*. arXiv preprint arXiv:1409.1259.  
[4] Pal, N. R., & Pal, S. K. (1993). *A review on image segmentation techniques*. Pattern recognition, 26(9), 1277-1294.  
[5] Badrinarayanan, V., Kendall, A., & Cipolla, R. (2017). *Segnet: A deep convolutional encoder-decoder architecture for image segmentation*. IEEE transactions on pattern analysis and machine intelligence, 39(12), 2481-2495.  
[6] Papineni, K., Roukos, S., Ward, T., & Zhu, W. J. (2002, July). *BLEU: a method for automatic evaluation of machine translation*. In Proceedings of the 40th annual meeting on association for computational linguistics (pp. 311-318). Association for Computational Linguistics.  
[7] Kingma, D. P., & Welling, M. (2013). *Auto-encoding variational bayes*. arXiv preprint arXiv:1312.6114.  
[8] https://en.wikipedia.org/wiki/Kullback%E2%80%93Leibler_divergence     
[9] Dumoulin, V., & Visin, F. (2016). *A guide to convolution arithmetic for deep learning*. arXiv preprint arXiv:1603.07285.  also: https://github.com/vdumoulin/conv_arithmetic  
[10] Jang, E., Gu, S., & Poole, B. (2016). *Categorical reparameterization with gumbel-softmax*. arXiv preprint arXiv:1611.01144.  
[11] https://en.wikipedia.org/wiki/Cluster_analysis  